## Calculate coherence for a pretrained model

The following workflow describes how to use gensim to calculate coherence measures for an LDA model that has already identified topics.

## Extract the top 10 terms from the term frequency table

In [1]:
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import re

#TopicTermFreq = pd.read_csv('/Users/dankoban/Documents/EM6575/mallet_command_line/ct_tidy_topics.csv')
TopicTermFreq = pd.read_csv('/Users/dankoban/Documents/EM6575/twitter/twitter_tidy_topics.csv')

num_topics = 50
num_top_terms = 10

topic_term_list = []
top_terms = []
for k in range(1, num_topics+1):    
    top_terms_topic_k = TopicTermFreq[TopicTermFreq['topic'] == (k-1)].sort_values('count', ascending = False)['term'].tolist()[0:num_top_terms]
    top_terms_topic_k = [re.sub(r'\W+', '', term) for term in top_terms_topic_k]    
    top_terms = top_terms + top_terms_topic_k
    topic_term_list.append(top_terms_topic_k)

top_terms = list(set(top_terms))

## Load the raw text files and parse to retain top term vocab only

In [ ]:
texts = []
counter = 0
with open('/Users/dankoban/Documents/CT_LDA/CT_data/mallet_input_data_crowdtangle.txt','r') as infile:
    for line in infile:
        line = line.split(' ')                  
        line = [re.sub(r'\W+', '', term) for term in line]
        line = [word.lower() for word in line if word.lower() in top_terms]
        line = list(set(line))
        counter += 1
        if counter %500000 == 0:            
            print(counter)
        texts.append(line)      

In [2]:
import os
input_dir = '/Users/dankoban/Documents/EM6575/twitter/hashtag model/hashtags'

# Extract file names from input directory
files = [file for file in os.listdir(input_dir) if file.endswith(".txt")]   
file_paths = [input_dir + "/" + file for file in files]
file_paths = file_paths

texts = []
counter = 0
for file in file_paths:
    with open(file,'r') as infile:    
        for line in infile:
            line = line.split(' ')                  
            line = [re.sub(r'\W+', '', term) for term in line]
            line = [word.lower() for word in line if word.lower() in top_terms]
            line = list(set(line))
            counter += 1
            if counter %500000 == 0:            
                print(counter)
            texts.append(line)

500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
10000000
10500000


In [3]:
len(texts)

10951065

## Transform the raw text into bag of words dictionary and corpus

In [ ]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

## Calculate topic coherence 

In [ ]:
from gensim.models import CoherenceModel

cm = CoherenceModel(topics=topic_term_list, corpus=corpus, dictionary=dictionary, coherence='u_mass')
coherence_scores = cm.get_coherence_per_topic()
coherence_scores